# Generate

Use an existing model to generate coefficients of a curve

In [1]:
import sagemaker
import torch
import torch.nn as nn
import torch.nn.functional as F

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [2]:

## This cell is only here as an example from another notebook.  No need to run this cell
from sagemaker.pytorch import PyTorch

git_config = {'repo': 'https://github.com/jcox22/Sagemaker_practice_gan.git'}

estimator = PyTorch(entry_point='training.py',
                    role=role,
                    framework_version='1.4.0',
                    py_version='py3',
                    git_config=git_config,
                    train_instance_count=1,
                    train_instance_type='ml.g4dn.xlarge')

estimator.fit()

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-05-18 20:02:05 Starting - Starting the training job...
2021-05-18 20:02:28 Starting - Launching requested ML instancesProfilerReport-1621368124: InProgress
......
2021-05-18 20:03:28 Starting - Preparing the instances for training.........
2021-05-18 20:04:49 Downloading - Downloading input data......
2021-05-18 20:05:49 Training - Downloading the training image...
2021-05-18 20:06:32 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-05-18 20:06:22,334 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-05-18 20:06:22,356 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-05-18 20:06:23,771 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-05-18 20:06:24,020 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating

In [7]:
# This is the cell for the estimator
from sagemaker.pytorch import PyTorch

git_config = {'repo': 'https://github.com/jcox22/Sagemaker_practice_gan.git'}

estimator = PyTorch(entry_point='generate.py',
                    role=role,
                    framework_version='1.4.0',
                    py_version='py3',
                    git_config=git_config,
                    train_instance_count=1,
                    train_instance_type='local')

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [8]:
# Here I was trying to make it local
from sagemaker.local import LocalSession

sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}



SyntaxError: invalid syntax (<ipython-input-8-980179b3a4fc>, line 6)

In [10]:
# Another attempt at local (bash, won't run as is)
SAGEMAKER_NETWORK = `docker network ls | grep -c sagemaker-local`
if [ $SAGEMAKER_NETWORK -eq 0 ]; then
  docker network create --driver bridge sagemaker-local
fi

SyntaxError: invalid syntax (<ipython-input-10-f0fee4afdf13>, line 2)

In [6]:
# Trying to deploy the model, will start running when you replace 'local'
from sagemaker.pytorch import PyTorchModel

# Below code fixes an issue with Estimator not being associated with a training job
#training_job_name = estimator.latest_training_job.name
#attached_estimator = Estimator.attach(training_job_name)
#attached_estimator.logs()
#attached_estimator.deploy()


estimator = sagemaker.estimator.Estimator.attach('pytorch-training-2021-05-18-19-03-48-042')

coefficients = estimator.deploy(initial_instance_count=1, instance_type='local')


2021-05-18 19:08:53 Starting - Preparing the instances for training
2021-05-18 19:08:53 Downloading - Downloading input data
2021-05-18 19:08:53 Training - Training image download completed. Training in progress.
2021-05-18 19:08:53 Uploading - Uploading generated training model
2021-05-18 19:08:53 Completed - Training job completed


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: 1 validation error detected: Value 'local' at 'productionVariants.1.member.instanceType' failed to satisfy constraint: Member must satisfy enum value set: [ml.r5d.12xlarge, ml.r5.12xlarge, ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.r5d.24xlarge, ml.r5.24xlarge, ml.p3.16xlarge, ml.m5d.xlarge, ml.m5.large, ml.t2.xlarge, ml.p2.16xlarge, ml.m5d.12xlarge, ml.inf1.2xlarge, ml.m5d.24xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c4.4xlarge, ml.inf1.6xlarge, ml.c5d.2xlarge, ml.c5.4xlarge, ml.g4dn.xlarge, ml.g4dn.12xlarge, ml.c5d.4xlarge, ml.g4dn.2xlarge, ml.c4.8xlarge, ml.c4.large, ml.c5d.xlarge, ml.c5.large, ml.g4dn.4xlarge, ml.c5.9xlarge, ml.g4dn.16xlarge, ml.c5d.large, ml.c5.xlarge, ml.c5d.9xlarge, ml.c4.xlarge, ml.inf1.xlarge, ml.g4dn.8xlarge, ml.inf1.24xlarge, ml.m5d.2xlarge, ml.t2.2xlarge, ml.c5d.18xlarge, ml.m5d.4xlarge, ml.t2.medium, ml.c5.18xlarge, ml.r5d.2xlarge, ml.r5.2xlarge, ml.p3.

In [ ]:
# Outputs
batch_size = 16
input_length = 20
k = 10

noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
    
outputs = coefficients.predict(noise)